In [0]:
import requests
import pandas as pd
from datetime import datetime

In [0]:
def ExtrairDadosBitcoin():
    url = 'https://api.coinbase.com/v2/prices/spot'
    resposta = requests.get(url)
    return resposta.json()

In [0]:
def ExtrairCotacaoUsdBrl():
    chave_api = dbutils.widgets.get("chave_api")
    url = f'https://api.currencyfreaks.com/v2.0/rates/latest?apikey={chave_api}'
    resposta = requests.get(url)
    return resposta.json()

In [0]:
def tratar_dados_bitcoin(dados_json, taxa_usd_brl):
    ValorUsd = float (dados_json['data']['amount'])
    
    CriptoMoeda = dados_json['data']['base']
    MoedaOriginal = dados_json['data']['currency']
    
    ValorBrl = ValorUsd * taxa_usd_brl

    timestamp = datetime.now()

    DadosTratados = [{
      "ValorUsd": ValorUsd,
      "ValorBrl": ValorBrl,
      "CriptoMoeda": CriptoMoeda,
      "MoedaOriginal": MoedaOriginal,
      "taxa_convercao_usd_brl": taxa_usd_brl,
      "timestamp": timestamp
    }]
    return DadosTratados

In [0]:
DadosBitcoin = ExtrairDadosBitcoin()
DadosCotacao = ExtrairCotacaoUsdBrl()

taxa_usd_brl = float(DadosCotacao['rates']['BRL'])

dados_bitcoins_tratados = tratar_dados_bitcoin(DadosBitcoin, taxa_usd_brl)

print(dados_bitcoins_tratados)

In [0]:
df_pandas = pd.DataFrame(dados_bitcoins_tratados)

print (df_pandas)

In [0]:
df_spark = spark.createDataFrame(df_pandas)
df_spark.printSchema()

In [0]:
df_spark.write.format("delta").mode("append").saveAsTable("pipeline_api_bitcoin.bitcoin_data.dados_bitcoin")